## Setup

### Note

This is a function Notebook example in the DazzaJi Automated-Prompt-Engineering Repo at: https://github.com/dazzaji/Automated-Prompt-Engineering/blob/main/notebooks/User_Prompt_Automated_Engineering_Notebook_01.ipynb

In [ ]:
from IPython.display import Markdown

In [ ]:
!pip install openai cohere tiktoken prettytable

## Import Necessary Libraries

In [ ]:
import openai

In [ ]:

import getpass

# Get the OpenAI API key from the user
OPENAI_API_KEY = getpass.getpass(prompt='Please enter your OpenAI API Key: ')


## User Prompt Generation with GPT-4 and Testing with GPT-3.5

In [ ]:
def gather_user_inputs():
    explanation = input("Please provide a high-level explanation of what you want to accomplish with the prompt: ")
    num_prompts = int(input("Enter the number of user prompts you'd like GPT-4 to generate and test with GPT-3.5: "))
    return explanation, num_prompts

In [ ]:

def display_results_markdown(prompts, responses):
    output_lines = []
    for prompt, response in zip(prompts, responses):
        output_lines.append(f"- **User Prompt:** {prompt}")
        output_lines.append(f"  - **GPT-3.5 Output:** {response}")
    display(Markdown('\n'.join(output_lines)))


In [ ]:

def generate_user_prompts_with_api(explanation, num_prompts):
    # Explicitly make API calls to the v1/chat/completions endpoint for GPT-4
    prompts = []
    endpoint_url = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {OPENAI_API_KEY}",
        "Content-Type": "application/json",
        "User-Agent": "openai-python"
    }

    for _ in range(num_prompts):
        data = {
            "model": "gpt-4",
            "messages": [
                {"role": "user", "content": f"Please generate a prompt that, when presented to a language model, would likely produce an answer describing the following: {explanation}. Do not provide the answer, only the prompt."}
            ]
        }
        response = requests.post(endpoint_url, headers=headers, json=data)
        response_json = response.json()
        if "choices" in response_json:
            message_content = response_json["choices"][0]["message"]["content"]
            prompts.append(message_content.strip())
        else:
            raise ValueError(f"Unexpected API response structure: {response_json}")

    return prompts


In [ ]:

import requests

def test_user_prompts_with_chat_api_explicit(explanation, prompts):
    # Explicitly make API calls to the v1/chat/completions endpoint for gpt-3.5-turbo
    responses = []
    endpoint_url = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {OPENAI_API_KEY}",
        "Content-Type": "application/json",
        "User-Agent": "openai-python"
    }

    for prompt in prompts:
        data = {
            "model": "gpt-3.5-turbo",
            "messages": [
                {"role": "user", "content": f"The user's goal is {explanation}. Please help the user evaluate the clarity and effectiveness of the following prompt. Assess the quality of the prompt based on its likelihood of achieving the user's stated goal. Prompt: {prompt}"}
            ]
        }
        response = requests.post(endpoint_url, headers=headers, json=data)
        response_json = response.json()
        message_content = response_json["choices"][0]["message"]["content"]
        responses.append(message_content.strip())
    return responses


In [ ]:

def main_markdown():
    try:
        print("Gathering User Inputs...")
        explanation, num_prompts = gather_user_inputs()
        print(f"Explanation: {explanation}")
        print(f"Number of Prompts: {num_prompts}")

        print("Generating User Prompts with GPT-4...")
        prompts = generate_user_prompts_with_api(explanation, num_prompts)
        print(f"Generated Prompts: {prompts}")

        print("Testing User Prompts with GPT-3.5-turbo...")
        responses = test_user_prompts_with_chat_api_explicit(explanation, prompts)
        print(f"Responses: {responses}")

        print("Displaying Results...")
        display_results_markdown(prompts, responses)
        print("Process Completed Successfully!")

    except Exception as e:
        print("Error Encountered!")
        print(str(e))


To run the workflow, simply execute the cell below.

In [ ]:
main_markdown()